<a href="https://colab.research.google.com/github/Sibaso/Project3_quan.nt173312/blob/main/Multilingual_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 8.8MB/s 
     |████████████████████████████████| 890kB 16.0MB/s 
     |████████████████████████████████| 2.9MB 47.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=879ecceba60d26103780a4feaec89f07ea6ae95ebec2dc4dda42b3a82c0897f2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from sklearn.model_selection import train_test_split


def split_data(path):
    with open('/content/drive/My Drive/Project3/data/%s.txt'%path, 'r', encoding='utf-8') as f:
        data = list(f.read().splitlines())
        train, test = train_test_split(data, train_size=0.85, shuffle=True, random_state=1)
        train, valid = train_test_split(train, train_size=0.85, shuffle=True, random_state=1)
        with open('/content/drive/My Drive/Project3/data/%s_train.txt'%path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(train))
        with open('/content/drive/My Drive/Project3/data/%s_valid.txt'%path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(valid))
        with open('/content/drive/My Drive/Project3/data/%s_test.txt'%path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(test))

split_data('eng-deu')

In [ ]:
import torchtext

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from collections import defaultdict
from copy import deepcopy
import numpy as np
import pandas as pd
import re
from torchtext.data import Field, BucketIterator, TabularDataset, Dataset, Example
from torchsummary import summary
import math
import copy
from copy import deepcopy
from torch.optim.optimizer import Optimizer
import os
import time
import nltk
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# device='cpu'

cuda


In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# MultilingualBert = BertModel.from_pretrained("bert-base-multilingual-cased")

In [ ]:
seq1 = "hello world"
seq2 = "xin chào thế giới"
encoded_input = tokenizer([seq1, seq2], return_tensors='pt', padding=True)
trg = encoded_input['input_ids']
print(trg)
print(encoded_input['attention_mask'])
N, trg_len = trg.shape
trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
    trg_len, trg_len
)
print(trg_mask)
output = MultilingualBert(input_ids=trg, attention_mask=encoded_input['attention_mask'])
encoded_input = tokenizer(seq2)
print(encoded_input)
print(tokenizer.decode(encoded_input["input_ids"]))
print(output.last_hidden_state.shape)
print(output.pooler_output.shape)
# print(output)

In [ ]:
encoded_input = tokenizer(eng_train[1])
print(encoded_input)
print(eng_train[1].split(' '))
print(tokenizer.decode(encoded_input["input_ids"]).split(' '))

{'input_ids': [101, 12277, 13028, 12796, 31862, 16683, 13172, 13028, 22469, 11783, 33264, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['if', 'you', 'worry', 'too', 'much', 'you', 'll', 'go', 'bald', '']
['[CLS]', 'if', 'you', 'worry', 'too', 'much', 'you', 'll', 'go', 'bald', '[SEP]']


In [ ]:
energy = torch.ones(2,3,3)
mask = torch.tril(torch.ones((3, 3))).expand(
    3, 3
)
print(mask)
energy = energy.masked_fill(mask == 0, float("-1e20"))
print(energy)
mask = torch.ones(3)
print(mask)
energy = torch.ones(3,3)
energy = energy.masked_fill(mask == 0, float("-1e20"))
print(energy)

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
tensor([[[ 1.0000e+00, -1.0000e+20, -1.0000e+20],
         [ 1.0000e+00,  1.0000e+00, -1.0000e+20],
         [ 1.0000e+00,  1.0000e+00,  1.0000e+00]],

        [[ 1.0000e+00, -1.0000e+20, -1.0000e+20],
         [ 1.0000e+00,  1.0000e+00, -1.0000e+20],
         [ 1.0000e+00,  1.0000e+00,  1.0000e+00]]])
tensor([1., 1., 1.])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])


In [ ]:

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.att_fc = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)  # (N, value_len, heads, head_dim)
        keys = self.keys(keys)  # (N, key_len, heads, head_dim)
        queries = self.queries(query)  # (N, query_len, heads, heads_dim)

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)

        # Mask padded indices so their weights become 0
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # attention shape: (N, heads, query_len, key_len)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.

        out = self.att_fc(out)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)
        return out


class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.trb_fc1 = nn.Linear(embed_size, forward_expansion * embed_size)
        self.relu = nn.ReLU()
        self.trb_fc2 = nn.Linear(forward_expansion * embed_size, embed_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.trb_fc1(x)
        forward = self.relu(forward)
        forward = self.trb_fc2(forward)
        out = self.dropout(self.norm2(forward + x))
        return out


class Encoder(nn.Module):
    def __init__(
        self,
        embed_size,
        num_layers,
        forward_expansion,
        heads,
        dropout,
        vocab_size,
        max_length
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(device)
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )
        # In the Encoder the query, key, value are all the same, it's in the
        # decoder this will change. This might look a bit odd in this case.
        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out


class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out


class Decoder(nn.Module):
    def __init__(
        self,
        embed_size,
        num_layers,
        forward_expansion,
        heads,
        dropout,
        vocab_size,
        max_length
    ):
        super(Decoder, self).__init__()
        self.embed_size = embed_size
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)

        return F.log_softmax(out, dim=-1)


class Transformer(nn.Module):
    def __init__(
        self,
        embed_size=768,
        num_layers=1,
        forward_expansion=4,
        heads=12,
        dropout=0.1,
        vocab_size=119547,
        max_length=512,
        pad_idx=0
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            embed_size,
            num_layers,
            forward_expansion,
            heads,
            dropout,
            vocab_size,
            max_length
        )

        self.decoder = Decoder(
            embed_size,
            num_layers,
            forward_expansion,
            heads,
            dropout,
            vocab_size,
            max_length
        )

        self.pad_idx = pad_idx

    def make_src_mask(self, src):
        src_mask = (src != self.pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out


In [ ]:

class EWC(object):
    """ Class implementing the Elastic Weight Consolidation approach described in http://arxiv.org/abs/1612.00796 """

    def __init__(self,model, lamb=4000, nepochs=100,sbatch=32,lr=0.001,lr_min=1e-6,lr_factor=3,lr_patience=5,name='model'):
        self.model=model
        self.model_old = deepcopy(self.get_param(self.model))
        self.fisher=None
        self.tasks = []

        self.nepochs = nepochs
        self.sbatch = sbatch
        self.lr = lr
        self.lr_min = lr_min
        self.lr_factor = lr_factor
        self.lr_patience = lr_patience
        self.lamb = lamb

        self.name = name


    def add_task(self, src_token, trg_token):
        if {'src': src_token, 'trg': trg_token} in self.tasks:
            return
        self.tasks.append({'src': src_token, 'trg': trg_token})
        
    def load_task(self, t):
        path = '/content/drive/My Drive/Project3/trained_model/{}|{}'.format(self.name,
                        '|'.join(['%s-%s'%(task['src'], task['trg']) for task in self.tasks[:t]]))
        self.model.load_state_dict(torch.load('%s.pt'%path))
        self.model.to(device)
        self.fisher = torch.load('%s_fisher.pt'%path)
        self.model_old = deepcopy(self.get_param(self.model))

    def get_param(self, model):
        sd = []
        for name, param in model.named_parameters():
            if 'word_embedding' in name or 'position_embedding' in name or 'norm' in name or 'norm1' in name or 'norm2' in name:
                continue 
            sd.append((name, param))
        return sd

    def get_optimizer(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.lr)
        # return torch.optim.SGD(self.model.parameters(), lr=self.lr, momentum=0.9)
   
    def train(self, src_train, trg_train, src_valid, trg_valid):
        best_loss = np.inf
        lr = self.lr
        patience = self.lr_patience
        self.ce=torch.nn.CrossEntropyLoss(ignore_index=self.model.pad_idx)
        self.optimizer=self.get_optimizer()
        path = '/content/drive/My Drive/Project3/trained_model/{}|{}'.format(self.name,
                        '|'.join(['%s-%s'%(task['src'], task['trg']) for task in self.tasks]))
        # Loop epochs
        for e in range(self.nepochs):
            try:
                # Train
                clock0=time.time()
                train_loss = self.train_epoch(src_train, trg_train)
                clock1=time.time()
                valid_loss = self.eval(src_valid, trg_valid)
                clock2=time.time()
                print('| Epoch {:3d}, time={:5.1f}ms/{:5.1f}ms | Train: loss={:.3f} |'.format(
                    e+1,1000*(clock1-clock0),
                    1000*(clock2-clock1),train_loss),end='')
                # # Valid
                print(' Valid: loss={:.3f} |'.format(valid_loss),end='')
                
                # Adapt lr
                if valid_loss < best_loss:
                    best_loss = valid_loss
                    torch.save(self.model.state_dict(),'%s.pt' % path)
                    patience = self.lr_patience
                    print(' *', end='')
                
                else:
                    patience -= 1
                    if patience <= 0:
                        lr /= self.lr_factor
                        print(' lr={:.1e}'.format(lr), end='')
                        if lr < self.lr_min:
                            print()
                            break
                            
                        patience = self.lr_patience
                        self.optimizer = self.get_optimizer()
                print()
            except KeyboardInterrupt:
                break

        # Restore best
        # torch.save(self.model.state_dict(),'%s.pt' % path)
        print('computing fisher ...')
        self.model.load_state_dict(torch.load('%s.pt'%path))
        self.model_old = deepcopy(self.get_param(self.model))
        
        # Fisher ops
        if len(self.tasks) > 1:
            fisher_old={}
            for n,_ in self.get_param(self.model):
                fisher_old[n]=self.fisher[n].clone()
        self.fisher=self.fisher_matrix_diag(src_train, trg_train)
        if len(self.tasks) > 1:
            t = len(self.tasks)-1
            # Watch out! We do not want to keep t models (or fisher diagonals) in memory, therefore we have to merge fisher diagonals
            for n,_ in self.get_param(self.model):
                self.fisher[n]=(self.fisher[n]+fisher_old[n]*t)/(t+1)       # Checked: it is better than the other option
                #self.fisher[n]=0.5*(self.fisher[n]+fisher_old[n])

        torch.save(self.fisher,'%s_fisher.pt'%path)
        return

    def train_epoch(self, src_data, trg_data):
        self.model.train()
        total_loss=0
        idx = np.random.permutation(src_data.shape[0])
        num_iters = src_data.shape[0]//self.sbatch+1
        for i in range(num_iters):
            top = min((i+1)*self.sbatch, src_data.shape[0])
            src = src_data[idx[i*self.sbatch:top]].to(device)
            trg = trg_data[idx[i*self.sbatch:top]].to(device)
            output = self.model(src, trg[:, :-1])
            output = output.reshape(-1, output.shape[2])
            trg = trg[:,1:].reshape(-1)   
            loss=self.criterion(output,trg)
            total_loss += loss.item()
            # Backward
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        return total_loss/num_iters

    def eval(self, src_data, trg_data):
        total_loss=0
        self.model.eval()
        idx = np.random.permutation(src_data.shape[0])
        num_iters = src_data.shape[0]//self.sbatch+1
        for i in range(num_iters):
            top = min((i+1)*self.sbatch, src_data.shape[0])
            src = src_data[idx[i*self.sbatch:top]].to(device)
            trg = trg_data[idx[i*self.sbatch:top]].to(device)
            output = self.model(src, trg[:, :-1])
            output = output.reshape(-1, output.shape[2])
            trg = trg[:,1:].reshape(-1)   
            loss=self.criterion(output,trg)
            total_loss += loss.item()

        return total_loss/num_iters

    def criterion(self,output,targets):
        # Regularization for all previous tasks
        loss_reg=0
        if len(self.tasks) > 1:
            for (name,param),(_,param_old) in zip(self.get_param(self.model),self.model_old):   
                # print(self.fisher[name].shape, param.shape, param_old.shape)
                loss_reg+=torch.sum(self.fisher[name]*(param_old-param).pow(2))/2
        return self.ce(output,targets)+self.lamb*loss_reg

    def fisher_matrix_diag(self, src_data, trg_data):
        # Init
        fisher={}
        for n,p in self.get_param(self.model):
            fisher[n]=0*p.data
        # Compute
        self.model.train()
        idx = np.random.permutation(src_data.shape[0])
        num_iters = src_data.shape[0]//self.sbatch+1
        for i in range(num_iters):
            top = min((i+1)*self.sbatch, src_data.shape[0])
            src = src_data[idx[i*self.sbatch:top]].to(device)
            trg = trg_data[idx[i*self.sbatch:top]].to(device)
            output = self.model(src, trg[:, :-1])
            output = output.reshape(-1, output.shape[2])
            trg = trg[:,1:].reshape(-1)
            loss=self.criterion(output,trg)
            loss.backward()
            # Get gradients
            for n,p in self.get_param(self.model):
                if p.grad is not None:
                    fisher[n]+=src.shape[0]*p.grad.data.pow(2)
        # Mean
        with torch.no_grad():
            for n,_ in self.get_param(self.model):
                fisher[n]=fisher[n]/num_iters
        return fisher

def translate(model, src_sen, max_length=512):

    model.eval()
    src_tensor = tokenizer(src_sen, return_tensors='pt')['input_ids'].to(device)
    outputs = [tokenizer.cls_token_id]
    for i in range(max_length):
        translate_tensor = torch.LongTensor(outputs).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(src_tensor, translate_tensor)
        
        best_guess = output.argmax(2)[:,-1].item()
        outputs.append(best_guess)

        if best_guess == tokenizer.sep_token_id:
            break

    outputs = outputs[1:-1]
    translated_sentence = tokenizer.decode(outputs)
    return translated_sentence

def eval_bleu_score(model, src_data, trg_data):
    total_bleu = 0
    for src_sen, trg_sen in zip(src_data, trg_data):
        translated_sentence = translate(model, src_sen)
        bleu_score = nltk.translate.bleu_score.sentence_bleu([trg_sen.split()], translated_sentence.split(), weights=(0.5, 0.5))
        total_bleu += bleu_score
    return total_bleu/len(src_data)

In [ ]:
model = Transformer(
    embed_size=768,
    num_layers=1,
    forward_expansion=4,
    heads=12,
    dropout=0.1,
    vocab_size=tokenizer.vocab_size,
    max_length=512,
    pad_idx=0
).to(device)

ewc = EWC(
    model=model,
    lamb=4000,
    nepochs=20,
    sbatch=32,
    lr=0.001,
    lr_min=1e-6,
    lr_factor=3,
    lr_patience=5,
    name='mult_bert'
)

In [ ]:
def preprocess(text):
    # text = ViTokenizer.tokenize(text)
    return ' '.join(re.split('\W+', text)).lower()

def load_data(path):
    with open('/content/drive/MyDrive/Project3/data/%s'%path, encoding='utf-8') as f:
        data = [line.split('\t')[:2] for line in f.read().splitlines()]

    src = [preprocess(line[0]) for line in data]
    trg = [preprocess(line[1]) for line in data]
    return src, trg

eng_train, viet_train = load_data('eng-viet_train.txt')
eng_train_tensor = tokenizer(eng_train, return_tensors='pt', padding=True)['input_ids']
viet_train_tensor = tokenizer(viet_train, return_tensors='pt', padding=True)['input_ids']

eng_valid, viet_valid = load_data('eng-viet_valid.txt')
eng_valid_tensor = tokenizer(eng_valid, return_tensors='pt', padding=True)['input_ids']
viet_valid_tensor = tokenizer(viet_valid, return_tensors='pt', padding=True)['input_ids']

eng_test, viet_test = load_data('eng-viet_test.txt')

print('train:', len(eng_train))
print('valid:', len(eng_valid))
print('test:', len(eng_test))

train: 2949
valid: 521
test: 613


In [ ]:
ewc.add_task('viet','eng')
ewc.train(viet_train_tensor, eng_train_tensor, viet_valid_tensor, eng_valid_tensor)

| Epoch   1, time=31969.9ms/1239.3ms | Train: loss=6.026 | Valid: loss=4.980 | *
| Epoch   2, time=32349.6ms/1272.9ms | Train: loss=4.092 | Valid: loss=4.195 | *
| Epoch   3, time=32885.3ms/1315.2ms | Train: loss=3.024 | Valid: loss=3.886 | *
| Epoch   4, time=33351.3ms/1353.5ms | Train: loss=2.179 | Valid: loss=3.711 | *
| Epoch   5, time=33945.7ms/1388.3ms | Train: loss=1.501 | Valid: loss=3.619 | *
| Epoch   6, time=34466.8ms/1403.0ms | Train: loss=0.967 | Valid: loss=3.740 |
| Epoch   7, time=34592.6ms/1403.1ms | Train: loss=0.638 | Valid: loss=3.722 |
| Epoch   8, time=34460.6ms/1413.0ms | Train: loss=0.435 | Valid: loss=3.884 |
| Epoch   9, time=34605.7ms/1412.5ms | Train: loss=0.297 | Valid: loss=3.910 |
| Epoch  10, time=34543.2ms/1396.0ms | Train: loss=0.223 | Valid: loss=4.016 | lr=3.3e-04
| Epoch  11, time=34513.8ms/1405.9ms | Train: loss=0.357 | Valid: loss=4.034 |
| Epoch  12, time=34504.2ms/1407.1ms | Train: loss=0.241 | Valid: loss=4.203 |
| Epoch  13, time=34495.4ms/140

In [ ]:
task1_bleu = eval_bleu_score(model, viet_test, eng_test)
print('task 1, test viet-eng:', task1_bleu)
task1_bleu = eval_bleu_score(model, viet_valid, eng_valid)
print('task 1, valid viet-eng:', task1_bleu)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


task 1, test viet-eng: 0.3998202204324897
task 1, valid viet-eng: 0.39602111879803276


In [ ]:
eng1_train, fra_train = load_data('eng-fra_train.txt')
eng1_train, fra_train = eng1_train[:5000], fra_train[:5000]
eng1_train_tensor = tokenizer(eng1_train, return_tensors='pt', padding=True)['input_ids']
fra_train_tensor = tokenizer(fra_train, return_tensors='pt', padding=True)['input_ids']

eng1_valid, fra_valid = load_data('eng-fra_valid.txt')
eng1_valid, fra_valid = eng1_valid[:600], fra_valid[:600]
eng1_valid_tensor = tokenizer(eng1_valid, return_tensors='pt', padding=True)['input_ids']
fra_valid_tensor = tokenizer(fra_valid, return_tensors='pt', padding=True)['input_ids']

eng1_test, fra_test = load_data('eng-fra_test.txt')
eng1_test, fra_test = eng1_test[:700], fra_test[:700]

print('train:', len(eng1_train))
print('valid:', len(eng1_valid))
print('test:', len(eng1_test))

train: 5000
valid: 600
test: 700


In [ ]:
ewc.add_task('fra','eng')
ewc.train(fra_train_tensor, eng1_train_tensor, fra_valid_tensor, eng1_valid_tensor)

| Epoch   1, time=65875.4ms/2057.6ms | Train: loss=8.423 | Valid: loss=4.952 | *
| Epoch   2, time=65891.3ms/2032.8ms | Train: loss=4.639 | Valid: loss=4.792 | *
| Epoch   3, time=65987.8ms/2046.4ms | Train: loss=4.237 | Valid: loss=4.719 | *
| Epoch   4, time=65940.9ms/2050.7ms | Train: loss=3.918 | Valid: loss=4.747 |
| Epoch   5, time=65822.7ms/2095.8ms | Train: loss=3.640 | Valid: loss=4.681 | *
| Epoch   6, time=66089.7ms/2085.7ms | Train: loss=3.418 | Valid: loss=4.680 | *
| Epoch   7, time=65993.3ms/2058.3ms | Train: loss=3.229 | Valid: loss=4.652 | *
| Epoch   8, time=66054.4ms/2066.5ms | Train: loss=3.070 | Valid: loss=4.674 |
| Epoch   9, time=66024.0ms/2096.5ms | Train: loss=2.939 | Valid: loss=4.635 | *
| Epoch  10, time=66100.8ms/2074.6ms | Train: loss=2.801 | Valid: loss=4.743 |
| Epoch  11, time=65986.9ms/2100.5ms | Train: loss=2.705 | Valid: loss=4.770 |
| Epoch  12, time=65924.5ms/2093.0ms | Train: loss=2.601 | Valid: loss=4.759 |
| Epoch  13, time=65943.3ms/2089.3ms |

In [ ]:
task1_bleu = eval_bleu_score(model, viet_test, eng_test)
print('task 1, test viet-eng:', task1_bleu)
task1_bleu = eval_bleu_score(model, viet_valid, eng_valid)
print('task 1, valid viet-eng:', task1_bleu)

task2_bleu = eval_bleu_score(model, fra_test, eng1_test)
print('task 2, test fra-eng:', task2_bleu)
task2_bleu = eval_bleu_score(model, fra_valid, eng1_valid)
print('task 2, valid fra-eng:', task2_bleu)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


task 1, test viet-eng: 0.37303431428479344
task 1, valid viet-eng: 0.3829520065195675
task 2, test fra-eng: 0.43069504524350344
task 2, valid fra-eng: 0.413542653300227


In [ ]:
eng2_train, deu_train = load_data('eng-deu_train.txt')
eng2_train, deu_train = eng2_train[:5000], deu_train[:5000]
eng2_train_tensor = tokenizer(eng2_train, return_tensors='pt', padding=True)['input_ids']
deu_train_tensor = tokenizer(deu_train, return_tensors='pt', padding=True)['input_ids']

eng2_valid, deu_valid = load_data('eng-deu_valid.txt')
eng2_valid, deu_valid = eng2_valid[:600], deu_valid[:600]
eng2_valid_tensor = tokenizer(eng2_valid, return_tensors='pt', padding=True)['input_ids']
deu_valid_tensor = tokenizer(deu_valid, return_tensors='pt', padding=True)['input_ids']

eng2_test, deu_test = load_data('eng-deu_test.txt')
eng2_test, deu_test = eng2_test[:700], deu_test[:700]

print('train:', len(eng2_train))
print('valid:', len(eng2_valid))
print('test:', len(eng2_test))

train: 5000
valid: 600
test: 700


In [ ]:
ewc.add_task('deu','eng')
ewc.train(deu_train_tensor, eng2_train_tensor, deu_valid_tensor, eng2_valid_tensor)

| Epoch   1, time=59841.8ms/1798.7ms | Train: loss=8.753 | Valid: loss=5.250 | *
| Epoch   2, time=59690.3ms/1806.1ms | Train: loss=4.834 | Valid: loss=4.953 | *
| Epoch   3, time=59503.6ms/1804.7ms | Train: loss=4.525 | Valid: loss=4.948 | *
| Epoch   4, time=59645.7ms/1804.1ms | Train: loss=4.333 | Valid: loss=4.895 | *
| Epoch   5, time=59672.1ms/1803.8ms | Train: loss=4.164 | Valid: loss=4.935 |
| Epoch   6, time=59297.3ms/1830.9ms | Train: loss=4.050 | Valid: loss=4.880 | *
| Epoch   7, time=59592.8ms/1824.8ms | Train: loss=3.940 | Valid: loss=4.951 |
| Epoch   8, time=59332.0ms/1823.5ms | Train: loss=3.860 | Valid: loss=4.880 | *
| Epoch   9, time=59636.9ms/1808.5ms | Train: loss=3.786 | Valid: loss=4.920 |
| Epoch  10, time=59363.7ms/1836.4ms | Train: loss=3.707 | Valid: loss=4.915 |
| Epoch  11, time=59350.8ms/1831.7ms | Train: loss=3.640 | Valid: loss=4.874 | *
| Epoch  12, time=59621.0ms/1804.4ms | Train: loss=3.581 | Valid: loss=4.926 |
| Epoch  13, time=59326.6ms/1829.1ms |

In [ ]:
task1_bleu = eval_bleu_score(model, viet_test, eng_test)
print('task 1, test viet-eng:', task1_bleu)
task1_bleu = eval_bleu_score(model, viet_valid, eng_valid)
print('task 1, valid viet-eng:', task1_bleu)

task2_bleu = eval_bleu_score(model, fra_test, eng1_test)
print('task 2, test fra-eng:', task2_bleu)
task2_bleu = eval_bleu_score(model, fra_valid, eng1_valid)
print('task 2, valid fra-eng:', task2_bleu)

task3_bleu = eval_bleu_score(model, deu_test, eng2_test)
print('task 3, test deu-eng:', task3_bleu)
task3_bleu = eval_bleu_score(model, deu_valid, eng2_valid)
print('task 3, valid deu-eng:', task3_bleu)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


task 1, test viet-eng: 0.3503040176663406
task 1, valid viet-eng: 0.35973018894940473
task 2, test fra-eng: 0.3954532992003429
task 2, valid fra-eng: 0.3835493491881809
task 3, test deu-eng: 0.4054799232042307
task 3, valid deu-eng: 0.3906546974230925


In [ ]:
eng3_train, cmn_train = load_data('eng-cmn_train.txt')
eng3_train, cmn_train = eng3_train[:5000], cmn_train[:5000]
eng3_train_tensor = tokenizer(eng3_train, return_tensors='pt', padding=True)['input_ids']
cmn_train_tensor = tokenizer(cmn_train, return_tensors='pt', padding=True)['input_ids']

eng3_valid, cmn_valid = load_data('eng-cmn_valid.txt')
eng3_valid, cmn_valid = eng3_valid[:600], cmn_valid[:600]
eng3_valid_tensor = tokenizer(eng3_valid, return_tensors='pt', padding=True)['input_ids']
cmn_valid_tensor = tokenizer(cmn_valid, return_tensors='pt', padding=True)['input_ids']

eng3_test, cmn_test = load_data('eng-cmn_test.txt')
eng3_test, cmn_test = eng3_test[:700], cmn_test[:700]

print('train:', len(eng3_train))
print('valid:', len(eng3_valid))
print('test:', len(eng3_test))

train: 5000
valid: 600
test: 700


In [ ]:
ewc.add_task('cmn','eng')
ewc.train(cmn_train_tensor, eng3_train_tensor, cmn_valid_tensor, eng3_valid_tensor)

| Epoch   1, time=52623.1ms/1721.3ms | Train: loss=10.103 | Valid: loss=5.305 | *
| Epoch   2, time=53195.8ms/1659.0ms | Train: loss=5.064 | Valid: loss=5.055 | *
| Epoch   3, time=53292.9ms/1680.8ms | Train: loss=4.700 | Valid: loss=4.817 | *
| Epoch   4, time=53196.2ms/1693.4ms | Train: loss=4.530 | Valid: loss=4.842 |
| Epoch   5, time=53084.6ms/1671.1ms | Train: loss=4.389 | Valid: loss=4.758 | *
| Epoch   6, time=53243.0ms/1677.2ms | Train: loss=4.295 | Valid: loss=4.750 | *
| Epoch   7, time=53208.0ms/1690.6ms | Train: loss=4.198 | Valid: loss=4.732 | *
| Epoch   8, time=53184.3ms/1686.4ms | Train: loss=4.131 | Valid: loss=4.712 | *
| Epoch   9, time=53269.5ms/1676.5ms | Train: loss=4.064 | Valid: loss=4.790 |
| Epoch  10, time=53059.7ms/1679.1ms | Train: loss=4.014 | Valid: loss=4.714 |
| Epoch  11, time=53160.3ms/1684.7ms | Train: loss=3.950 | Valid: loss=4.716 |
| Epoch  12, time=53162.9ms/1681.4ms | Train: loss=3.906 | Valid: loss=4.674 | *
| Epoch  13, time=53170.2ms/1671.0m

In [ ]:
task1_bleu = eval_bleu_score(model, viet_test, eng_test)
print('task 1, test viet-eng:', task1_bleu)
task1_bleu = eval_bleu_score(model, viet_valid, eng_valid)
print('task 1, valid viet-eng:', task1_bleu)

task2_bleu = eval_bleu_score(model, fra_test, eng1_test)
print('task 2, test fra-eng:', task2_bleu)
task2_bleu = eval_bleu_score(model, fra_valid, eng1_valid)
print('task 2, valid fra-eng:', task2_bleu)

task3_bleu = eval_bleu_score(model, deu_test, eng2_test)
print('task 3, test deu-eng:', task3_bleu)
task3_bleu = eval_bleu_score(model, deu_valid, eng2_valid)
print('task 3, valid deu-eng:', task3_bleu)

task4_bleu = eval_bleu_score(model, cmn_test, eng3_test)
print('task 4, test cmn-eng:', task4_bleu)
task4_bleu = eval_bleu_score(model, cmn_valid, eng3_valid)
print('task 4, valid cmn-eng:', task4_bleu)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


task 1, test viet-eng: 0.35527132347256246
task 1, valid viet-eng: 0.35280394367011453
task 2, test fra-eng: 0.38236695900386225
task 2, valid fra-eng: 0.371775317150316
task 3, test deu-eng: 0.38650534554786103
task 3, valid deu-eng: 0.3810107931110714
task 4, test cmn-eng: 0.3781235031608796
task 4, valid cmn-eng: 0.36552328419280294


In [ ]:
task4_bleu = eval_bleu_score(model, cmn_test, eng3_test)
print('task 4, test cmn-eng:', task4_bleu)
task4_bleu = eval_bleu_score(model, cmn_valid, eng3_valid)
print('task 4, valid cmn-eng:', task4_bleu)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


task 4, test cmn-eng: 0.3566312436051986
task 4, valid cmn-eng: 0.35761536666867244


In [ ]:
for i in np.random.choice(len(viet_test), 10, replace=False):
    src_sen = viet_test[i]
    trg_sen = eng_test[i]
    translated_sentence = translate(model, src_sen)
    bleu_score = nltk.translate.bleu_score.sentence_bleu([trg_sen.split()], translated_sentence.split(), weights=(0.5, 0.5))
    print()
    print('> source sentence:', src_sen)
    print('< translated sentence:', translated_sentence)
    print('= correct sentence:', trg_sen)
    print('bleu score:',bleu_score)


> source sentence: bạn có muốn biết tôi đã làm điều đó như thế nào không 
< translated sentence: do you want to do anything but i want to do anything but a job
= correct sentence: do you want to know how i did that 
bleu score: 0.26726124191242434

> source sentence: tôi không thích đùa kiểu này 
< translated sentence: i don t like this way
= correct sentence: i don t like these kinds of jokes 
bleu score: 0.45317419124773295

> source sentence: tôi đã ăn đồ tối qua cho bữa trưa 
< translated sentence: i ate ice cream at the concert lasted
= correct sentence: i ate last night s leftovers for lunch 
bleu score: 0.18898223650461357

> source sentence: làm ơn phô tô thêm vài bản nhé 
< translated sentence: please don t waste of yourself
= correct sentence: please have some copies made 
bleu score: 0.408248290463863


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



> source sentence: tôi bị đau bụng 
< translated sentence: i m glad i m diet
= correct sentence: i ve got an upset stomach 
bleu score: 0.408248290463863

> source sentence: sao bạn không tới thăm tụi tôi 
< translated sentence: why don t you give me to get up
= correct sentence: why don t you come visit us 
bleu score: 0.408248290463863

> source sentence: nhà vua đã bị xử tử 
< translated sentence: home just arrived
= correct sentence: the king was executed 
bleu score: 0

> source sentence: cô ấy là một người nói tiếng anh thông thạo 
< translated sentence: she was a famous composer from english
= correct sentence: she is a fluent speaker of english 
bleu score: 0.6546536707079771

> source sentence: tôi ngủ có 2 tiếng à hèn chi bây giờ buồn ngủ ghê 
< translated sentence: i sleeps venture of the largest yokohas and four of the intent
= correct sentence: i slept only two hours no wonder i m sleepy 
bleu score: 0.28867513459481287

> source sentence: quyền lực mang đến tham nhũng 
<

In [ ]:
for i in np.random.choice(len(eng1_test), 10, replace=False):
    src_sen = fra_test[i]
    trg_sen = eng1_test[i]
    translated_sentence = translate(model, src_sen)
    bleu_score = nltk.translate.bleu_score.sentence_bleu([trg_sen.split()], translated_sentence.split(), weights=(0.5, 0.5))
    print()
    print('> source sentence:', src_sen)
    print('< translated sentence:', translated_sentence)
    print('= correct sentence:', trg_sen)
    print('bleu score:',bleu_score)


> source sentence: je les ai laissées y aller 
< translated sentence: there are no one of the problem
= correct sentence: i let them go 
bleu score: 0

> source sentence: je les déteste tous 
< translated sentence: i love the whole world
= correct sentence: i hate them all 
bleu score: 0.447213595499958

> source sentence: ces chatons sont tellement mignons et câlins 
< translated sentence: these are engageds and physically these tickets are ninjas
= correct sentence: these kittens are so cute and cuddly 
bleu score: 0.5773502691896257

> source sentence: vous ne pourrez probablement pas faire cela 
< translated sentence: you don t make that for me to do that
= correct sentence: you probably won t be able to do that 
bleu score: 0.3333333333333333


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



> source sentence: vous aurez besoin de cela 
< translated sentence: you re need to do that
= correct sentence: you ll need that 
bleu score: 0.7071067811865476

> source sentence: tom gémit bruyamment 
< translated sentence: tom complaineding the track
= correct sentence: tom groaned loudly 
bleu score: 0.5

> source sentence: je ne pensais pas que vous étiez si vieux 
< translated sentence: i didn t think you weren t you weren
= correct sentence: i didn t think you were so old 
bleu score: 0.5270462766947299

> source sentence: tom a adoré le film 
< translated sentence: tom has changed the telephone
= correct sentence: tom enjoyed the movie 
bleu score: 0.6324555320336759

> source sentence: tom réalisa qu il était seul 
< translated sentence: tom was built it was arrogant
= correct sentence: tom realized he was alone 
bleu score: 0.5773502691896257

> source sentence: je suis dépourvu d idées 
< translated sentence: i m sorry of my ideas
= correct sentence: i m all out of ideas 
b

In [ ]:
for i in np.random.choice(len(deu_test), 10, replace=False):
    src_sen = deu_test[i]
    trg_sen = eng2_test[i]
    translated_sentence = translate(model, src_sen)
    bleu_score = nltk.translate.bleu_score.sentence_bleu([trg_sen.split()], translated_sentence.split(), weights=(0.5, 0.5))
    print()
    print('> source sentence:', src_sen)
    print('< translated sentence:', translated_sentence)
    print('= correct sentence:', trg_sen)
    print('bleu score:',bleu_score)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



> source sentence: tom spielte krank 
< translated sentence: tom has a new car
= correct sentence: tom pretended to be sick 
bleu score: 0.447213595499958

> source sentence: es macht tom nicht allzu viel aus 
< translated sentence: tom doesn t have any longer
= correct sentence: tom doesn t mind all that much 
bleu score: 0.3785581357133378

> source sentence: ich kann tom nicht allein lassen 
< translated sentence: i can t let tom leave
= correct sentence: i can t leave tom behind 
bleu score: 0.5773502691896258

> source sentence: japan betreibt viel handel mit kanada 
< translated sentence: japan has a lot of nerve
= correct sentence: japan does a lot of trade with canada 
bleu score: 0.3700151777184613

> source sentence: ihnen bleibt nur noch sehr wenig zeit 
< translated sentence: it s very so much time
= correct sentence: you have very little time left 
bleu score: 0.5773502691896257

> source sentence: ich habe nach dir gesucht 
< translated sentence: i ve been waiting for yo

In [ ]:
for i in np.random.choice(len(cmn_test), 10, replace=False):
    src_sen = cmn_test[i]
    trg_sen = eng3_test[i]
    translated_sentence = translate(model, src_sen)
    bleu_score = nltk.translate.bleu_score.sentence_bleu([trg_sen.split()], translated_sentence.split(), weights=(0.5, 0.5))
    print()
    print('> source sentence:', src_sen)
    print('< translated sentence:', translated_sentence)
    print('= correct sentence:', trg_sen)
    print('bleu score:',bleu_score)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



> source sentence: 一隻鳥正在樹上唱歌 
< translated sentence: a new road is on the floor
= correct sentence: a bird is singing in the tree 
bleu score: 0.6546536707079771

> source sentence: 美國在1492年被哥倫布發現 
< translated sentence: armenian is on the streets
= correct sentence: america was discovered by columbus in 1492 
bleu score: 0

> source sentence: 那孩子沒有什麼朋友 
< translated sentence: what scared you ve already begun
= correct sentence: that child has few friends 
bleu score: 0

> source sentence: 汤姆什么都没告诉你吗 
< translated sentence: tom didn t know what you said
= correct sentence: hasn t tom told you anything 
bleu score: 0.6546536707079771

> source sentence: 我能回答所有的問題 
< translated sentence: i can understand yourself
= correct sentence: i could answer all the questions 
bleu score: 0.3032653298563167

> source sentence: 我不完全肯定 
< translated sentence: i m not sure to be a candidate
= correct sentence: i m not absolutely sure 
bleu score: 0.3779644730092272

> source sentence: 这不可能 
< transla

In [ ]:
ewc.tasks=[]
ewc.add_task('viet','eng')
ewc.add_task('fra','eng')
ewc.add_task('deu','eng')
ewc.load_task(3)

In [ ]:
def preprocess(text):
    # text = ViTokenizer.tokenize(text)
    return ' '.join(re.split('\W+', text)).lower()

with open('/content/drive/MyDrive/Project3/data/MT-EV-VLSP2020/indomain-news/train.en', encoding='utf-8') as f:
    eng_train = [preprocess(line) for line in f.read().splitlines()]

with open('/content/drive/MyDrive/Project3/data/MT-EV-VLSP2020/indomain-news/train.vi', encoding='utf-8') as f:
    viet_train = [preprocess(line) for line in f.read().splitlines()]

with open('/content/drive/MyDrive/Project3/data/MT-EV-VLSP2020/indomain-news/tst.en', encoding='utf-8') as f:
    eng_test = [preprocess(line) for line in f.read().splitlines()]

with open('/content/drive/MyDrive/Project3/data/MT-EV-VLSP2020/indomain-news/tst.vi', encoding='utf-8') as f:
    viet_test = [preprocess(line) for line in f.read().splitlines()]

with open('/content/drive/MyDrive/Project3/data/MT-EV-VLSP2020/indomain-news/dev.en', encoding='utf-8') as f:
    eng_valid = [preprocess(line) for line in f.read().splitlines()]

with open('/content/drive/MyDrive/Project3/data/MT-EV-VLSP2020/indomain-news/dev.vi', encoding='utf-8') as f:
    viet_valid = [preprocess(line) for line in f.read().splitlines()]

eng_train_tensor = tokenizer(eng_train, return_tensors='pt', padding=True)['input_ids']
viet_train_tensor = tokenizer(viet_train, return_tensors='pt', padding=True)['input_ids']

eng_valid_tensor = tokenizer(eng_valid, return_tensors='pt', padding=True)['input_ids']
viet_valid_tensor = tokenizer(viet_valid, return_tensors='pt', padding=True)['input_ids']

print('train:', len(eng_train))
print('valid:', len(eng_valid))
print('test:', len(eng_test))

train: 20000
valid: 1007
test: 1220
